In [1]:
import pickle
import math
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import VotingClassifier

class EnsembleModel:
    def __init__(self):
        with open("../model/ensemble_model.pkl", "rb") as f:
            ensemble_model = pickle.load(f)
        with open("../scaler/scaler0924.pkl", "rb") as f:
            scaler = pickle.load(f)
        self.model: VotingClassifier = ensemble_model
        self.scaler: MinMaxScaler = scaler

    def predict(self, keypoint_3d):
        preprocessed_data = self.__preprocess(keypoint_3d)
        print(preprocessed_data.shape)
        return self.model.predict(preprocessed_data)

    def __preprocess(self, keypoint_3d):
        if self.scaler is None:
            raise ValueError("Scaler not loaded.")
        data = np.array([np.array([d["x"], d["y"], d["z"]]) for d in keypoint_3d])

        # 엉덩이를 기준으로 좌표 재설정
        hip_coords = data[23]
        for i in range(data.shape[0]):
            data[i] -= hip_coords

        # 팔 각도 계산
        shoulder_right = data[11]
        elbow_right = data[13]
        wrist_right = data[15]

        shoulder_left = data[12]
        elbow_left = data[14]
        wrist_left= data[16]

        angle_right_arm = self.__calculate_angle(shoulder_right, elbow_right, wrist_right)
        angle_left_arm = self.__calculate_angle(shoulder_left, elbow_left, wrist_left)

        xy_angle_right_arm = self.__calculate_xy_angle(shoulder_right[:-1], elbow_right[:-1], wrist_right[:-1])
        xy_angle_left_arm = self.__calculate_xy_angle(shoulder_left[:-1], elbow_left[:-1], wrist_left[:-1])

        data = np.append(data, round(angle_right_arm, 2))
        data = np.append(data, round(angle_left_arm, 2))
        data = np.append(data, round(xy_angle_right_arm, 2))
        data = np.append(data, round(xy_angle_left_arm, 2))

        normalized_data = self.scaler.transform([data])
        preprocessed_data = normalized_data[0][-4:]

        return [preprocessed_data] # 2차원 배열로 반환

    def __calculate_angle(self, a, b, c):
        ba = a - b # vector from point b to a
        bc = c - b # vector from point b to c

        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(cosine_angle)

        return np.degrees(angle)

    def __calculate_xy_angle(self, a, b, c):
        # 벡터 생성
        ba = [a[0]-b[0], a[1]-b[1]] # 벡터 BA
        bc = [c[0]-b[0], c[1]-b[1]] # 벡터 BC

        # 내적 계산
        dot_product = ba[0]*bc[0] + ba[1]*bc[1]

        # 두 벡터의 크기 계산
        magnitude_ba = math.sqrt(ba[0]**2 + ba[1]**2)
        magnitude_bc = math.sqrt(bc[0]**2 + bc[1]**2)

        # cos(theta) 계산
        cos_theta = dot_product / (magnitude_ba * magnitude_bc)

        # acos(cos_theta)를 사용하여 theta(라디안 단위) 찾기, 그리고 degree로 변환 
        angle_in_degree = math.acos(cos_theta) * (180 / math.pi)

        return angle_in_degree

In [2]:
keypoint_3d = [
    {
        "x": -0.053819116204977036,
        "y": -0.5335278511047363,
        "z": -0.25063684582710266,
        "score": 0.9994493722915649,
        "name": "nose"
    },
    {
        "x": -0.03560296446084976,
        "y": -0.5737643241882324,
        "z": -0.25775232911109924,
        "score": 0.9987234473228455,
        "name": "left_eye_inner"
    },
    {
        "x": -0.03552299365401268,
        "y": -0.5755583643913269,
        "z": -0.25769707560539246,
        "score": 0.998975396156311,
        "name": "left_eye"
    },
    {
        "x": -0.035329703241586685,
        "y": -0.5757691264152527,
        "z": -0.2574390172958374,
        "score": 0.9986988306045532,
        "name": "left_eye_outer"
    },
    {
        "x": -0.06044427305459976,
        "y": -0.5707638263702393,
        "z": -0.2406347692012787,
        "score": 0.9989168643951416,
        "name": "right_eye_inner"
    },
    {
        "x": -0.05997969210147858,
        "y": -0.5712913274765015,
        "z": -0.24305008351802826,
        "score": 0.9991714358329773,
        "name": "right_eye"
    },
    {
        "x": -0.060824327170848846,
        "y": -0.5718116164207458,
        "z": -0.2417135238647461,
        "score": 0.999062716960907,
        "name": "right_eye_outer"
    },
    {
        "x": 0.05527786538004875,
        "y": -0.5803398489952087,
        "z": -0.20849204063415527,
        "score": 0.9990860223770142,
        "name": "left_ear"
    },
    {
        "x": -0.05865207687020302,
        "y": -0.5781159996986389,
        "z": -0.14155231416225433,
        "score": 0.9993451237678528,
        "name": "right_ear"
    },
    {
        "x": -0.01875346526503563,
        "y": -0.5200114846229553,
        "z": -0.23006851971149445,
        "score": 0.9995546340942383,
        "name": "mouth_left"
    },
    {
        "x": -0.051989056169986725,
        "y": -0.5145512819290161,
        "z": -0.20968051254749298,
        "score": 0.9995325803756714,
        "name": "mouth_right"
    },
    {
        "x": 0.1800168752670288,
        "y": -0.4178190231323242,
        "z": -0.1062871590256691,
        "score": 0.9951345324516296,
        "name": "left_shoulder"
    },
    {
        "x": -0.141890749335289,
        "y": -0.4395427405834198,
        "z": -0.09479015320539474,
        "score": 0.9993962645530701,
        "name": "right_shoulder"
    },
    {
        "x": 0.27205345034599304,
        "y": -0.2162456065416336,
        "z": -0.0784779042005539,
        "score": 0.12770254909992218,
        "name": "left_elbow"
    },
    {
        "x": -0.23056916892528534,
        "y": -0.22575607895851135,
        "z": -0.04066173732280731,
        "score": 0.19650980830192566,
        "name": "right_elbow"
    },
    {
        "x": 0.2398059219121933,
        "y": -0.04915419593453407,
        "z": -0.09695281088352203,
        "score": 0.008585016243159771,
        "name": "left_wrist"
    },
    {
        "x": -0.23927980661392212,
        "y": -0.07858394831418991,
        "z": -0.10453706979751587,
        "score": 0.03524139150977135,
        "name": "right_wrist"
    },
    {
        "x": 0.23605358600616455,
        "y": 0.01569535955786705,
        "z": -0.0886107012629509,
        "score": 0.012041179463267326,
        "name": "left_pinky"
    },
    {
        "x": -0.23376724123954773,
        "y": -0.01916261576116085,
        "z": -0.12013287842273712,
        "score": 0.04165929555892944,
        "name": "right_pinky"
    },
    {
        "x": 0.2110181748867035,
        "y": -0.012674044817686081,
        "z": -0.11577658355236053,
        "score": 0.020828597247600555,
        "name": "left_index"
    },
    {
        "x": -0.1902133673429489,
        "y": -0.039940547198057175,
        "z": -0.14155812561511993,
        "score": 0.06632979214191437,
        "name": "right_index"
    },
    {
        "x": 0.22003032267093658,
        "y": -0.0511900819838047,
        "z": -0.09993761032819748,
        "score": 0.02254684641957283,
        "name": "left_thumb"
    },
    {
        "x": -0.21385633945465088,
        "y": -0.07329989969730377,
        "z": -0.12100441008806229,
        "score": 0.06489181518554688,
        "name": "right_thumb"
    },
    {
        "x": 0.1337704360485077,
        "y": 0.014303861185908318,
        "z": 0.0015255078906193376,
        "score": 0.0004318389401305467,
        "name": "left_hip"
    },
    {
        "x": -0.13322922587394714,
        "y": -0.03611418232321739,
        "z": 0.00378307793289423,
        "score": 0.0006214262684807181,
        "name": "right_hip"
    },
    {
        "x": 0.105281300842762,
        "y": 0.015234251506626606,
        "z": -0.11410687118768692,
        "score": 0.00037839566357433796,
        "name": "left_knee"
    },
    {
        "x": -0.04016163572669029,
        "y": -0.28218409419059753,
        "z": -0.18665452301502228,
        "score": 0.00028762980946339667,
        "name": "right_knee"
    },
    {
        "x": 0.17697307467460632,
        "y": 0.3140511214733124,
        "z": -0.0025348274502903223,
        "score": 0.00005724634320358746,
        "name": "left_ankle"
    },
    {
        "x": -0.01883940026164055,
        "y": 0.03015322983264923,
        "z": -0.15828919410705566,
        "score": 0.00001196182165585924,
        "name": "right_ankle"
    },
    {
        "x": 0.16883018612861633,
        "y": 0.3758017122745514,
        "z": 0.03229289501905441,
        "score": 0.00005742647044826299,
        "name": "left_heel"
    },
    {
        "x": -0.04341854527592659,
        "y": 0.0904620885848999,
        "z": -0.08297878503799438,
        "score": 0.00003318589006084949,
        "name": "right_heel"
    },
    {
        "x": 0.2807851731777191,
        "y": 0.05365189164876938,
        "z": -0.31145375967025757,
        "score": 0.000060156518884468824,
        "name": "left_foot_index"
    },
    {
        "x": -0.015509415417909622,
        "y": -0.19010253250598907,
        "z": -0.4606517553329468,
        "score": 0.000049115216825157404,
        "name": "right_foot_index"
    }
]

In [103]:
model = EnsembleModel()
model.predict(keypoint_3d), model.summary()

data [-1.87589552e-01 -5.47831712e-01 -2.52162354e-01 -1.69373401e-01
 -5.88068185e-01 -2.59277837e-01 -1.69293430e-01 -5.89862226e-01
 -2.59222583e-01 -1.69100139e-01 -5.90072988e-01 -2.58964525e-01
 -1.94214709e-01 -5.85067688e-01 -2.42160277e-01 -1.93750128e-01
 -5.85595189e-01 -2.44575591e-01 -1.94594763e-01 -5.86115478e-01
 -2.43239032e-01 -7.84925707e-02 -5.94643710e-01 -2.10017549e-01
 -1.92422513e-01 -5.92419861e-01 -1.43077822e-01 -1.52523901e-01
 -5.34315346e-01 -2.31594028e-01 -1.85759492e-01 -5.28855143e-01
 -2.11206020e-01  4.62464392e-02 -4.32122884e-01 -1.07812667e-01
 -2.75661185e-01 -4.53846602e-01 -9.63156611e-02  1.38283014e-01
 -2.30549468e-01 -8.00034121e-02 -3.64339605e-01 -2.40059940e-01
 -4.21872452e-02  1.06035486e-01 -6.34580571e-02 -9.84783188e-02
 -3.73050243e-01 -9.28878095e-02 -1.06062578e-01  1.02283150e-01
  1.39149837e-03 -9.01362092e-02 -3.67537677e-01 -3.34664769e-02
 -1.21658386e-01  7.72477388e-02 -2.69779060e-02 -1.17302091e-01
 -3.23983803e-01 -5.

/Users/hou27/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/Users/hou27/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/Users/hou27/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/hou27/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


TypeError: 'VotingClassifier' object is not subscriptable

In [6]:
keypoint_3d[0]["x"]

-0.053819116204977036

In [45]:
import numpy as np

data = np.array([np.array([d["x"], d["y"], d["z"]]) for d in keypoint_3d])

In [46]:
data

array([[-0.05381912, -0.53352785, -0.25063685],
       [-0.03560296, -0.57376432, -0.25775233],
       [-0.03552299, -0.57555836, -0.25769708],
       [-0.0353297 , -0.57576913, -0.25743902],
       [-0.06044427, -0.57076383, -0.24063477],
       [-0.05997969, -0.57129133, -0.24305008],
       [-0.06082433, -0.57181162, -0.24171352],
       [ 0.05527787, -0.58033985, -0.20849204],
       [-0.05865208, -0.578116  , -0.14155231],
       [-0.01875347, -0.52001148, -0.23006852],
       [-0.05198906, -0.51455128, -0.20968051],
       [ 0.18001688, -0.41781902, -0.10628716],
       [-0.14189075, -0.43954274, -0.09479015],
       [ 0.27205345, -0.21624561, -0.0784779 ],
       [-0.23056917, -0.22575608, -0.04066174],
       [ 0.23980592, -0.0491542 , -0.09695281],
       [-0.23927981, -0.07858395, -0.10453707],
       [ 0.23605359,  0.01569536, -0.0886107 ],
       [-0.23376724, -0.01916262, -0.12013288],
       [ 0.21101817, -0.01267404, -0.11577658],
       [-0.19021337, -0.03994055, -0.141

In [47]:
data.shape

(33, 3)

In [48]:
hip_coords = data[23]

# Subtract hip coordinates from all other coordinates
for i in range(data.shape[0]):
    data[i] -= hip_coords

# Now replace the original DataFrame with the new values
data

array([[-0.18758955, -0.54783171, -0.25216235],
       [-0.1693734 , -0.58806819, -0.25927784],
       [-0.16929343, -0.58986223, -0.25922258],
       [-0.16910014, -0.59007299, -0.25896453],
       [-0.19421471, -0.58506769, -0.24216028],
       [-0.19375013, -0.58559519, -0.24457559],
       [-0.19459476, -0.58611548, -0.24323903],
       [-0.07849257, -0.59464371, -0.21001755],
       [-0.19242251, -0.59241986, -0.14307782],
       [-0.1525239 , -0.53431535, -0.23159403],
       [-0.18575949, -0.52885514, -0.21120602],
       [ 0.04624644, -0.43212288, -0.10781267],
       [-0.27566119, -0.4538466 , -0.09631566],
       [ 0.13828301, -0.23054947, -0.08000341],
       [-0.3643396 , -0.24005994, -0.04218725],
       [ 0.10603549, -0.06345806, -0.09847832],
       [-0.37305024, -0.09288781, -0.10606258],
       [ 0.10228315,  0.0013915 , -0.09013621],
       [-0.36753768, -0.03346648, -0.12165839],
       [ 0.07724774, -0.02697791, -0.11730209],
       [-0.3239838 , -0.05424441, -0.143

In [49]:
import math

def calculate_angle(a, b, c):
    ba = a - b # vector from point b to a
    bc = c - b # vector from point b to c

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    return np.degrees(angle)

def calculate_xy_angle(a, b, c):
    # 벡터 생성
    ba = [a[0]-b[0], a[1]-b[1]] # 벡터 BA
    bc = [c[0]-b[0], c[1]-b[1]] # 벡터 BC

    # 내적 계산
    dot_product = ba[0]*bc[0] + ba[1]*bc[1]

    # 두 벡터의 크기 계산
    magnitude_ba = math.sqrt(ba[0]**2 + ba[1]**2)
    magnitude_bc = math.sqrt(bc[0]**2 + bc[1]**2)

    # cos(theta) 계산
    cos_theta = dot_product / (magnitude_ba * magnitude_bc)

    # acos(cos_theta)를 사용하여 theta(라디안 단위) 찾기, 그리고 degree로 변환 
    angle_in_degree = math.acos(cos_theta) * (180 / math.pi)

    return angle_in_degree

In [50]:
shoulder_right = data[11]
elbow_right = data[13]
wrist_right = data[15]

shoulder_left = data[12]
elbow_left = data[14]
wrist_left= data[16]

angle_right_arm= calculate_angle(shoulder_right, elbow_right, wrist_right)
angle_left_arm= calculate_angle(shoulder_left, elbow_left, wrist_left)

print(angle_right_arm)

xy_angle_right_arm= calculate_xy_angle(shoulder_right[:-1], elbow_right[:-1], wrist_right[:-1])
xy_angle_left_arm= calculate_xy_angle(shoulder_left[:-1], elbow_left[:-1], wrist_left[:-1])

data = np.append(data, round(angle_right_arm, 2))
print(data)
data = np.append(data, round(angle_left_arm, 2))
data = np.append(data, round(xy_angle_right_arm, 2))
data = np.append(data, round(xy_angle_left_arm, 2))

142.1833235022152
[-1.87589552e-01 -5.47831712e-01 -2.52162354e-01 -1.69373401e-01
 -5.88068185e-01 -2.59277837e-01 -1.69293430e-01 -5.89862226e-01
 -2.59222583e-01 -1.69100139e-01 -5.90072988e-01 -2.58964525e-01
 -1.94214709e-01 -5.85067688e-01 -2.42160277e-01 -1.93750128e-01
 -5.85595189e-01 -2.44575591e-01 -1.94594763e-01 -5.86115478e-01
 -2.43239032e-01 -7.84925707e-02 -5.94643710e-01 -2.10017549e-01
 -1.92422513e-01 -5.92419861e-01 -1.43077822e-01 -1.52523901e-01
 -5.34315346e-01 -2.31594028e-01 -1.85759492e-01 -5.28855143e-01
 -2.11206020e-01  4.62464392e-02 -4.32122884e-01 -1.07812667e-01
 -2.75661185e-01 -4.53846602e-01 -9.63156611e-02  1.38283014e-01
 -2.30549468e-01 -8.00034121e-02 -3.64339605e-01 -2.40059940e-01
 -4.21872452e-02  1.06035486e-01 -6.34580571e-02 -9.84783188e-02
 -3.73050243e-01 -9.28878095e-02 -1.06062578e-01  1.02283150e-01
  1.39149837e-03 -9.01362092e-02 -3.67537677e-01 -3.34664769e-02
 -1.21658386e-01  7.72477388e-02 -2.69779060e-02 -1.17302091e-01
 -3.239

In [26]:
data

array([-1.87589552e-01, -5.47831712e-01, -2.52162354e-01, -1.69373401e-01,
       -5.88068185e-01, -2.59277837e-01, -1.69293430e-01, -5.89862226e-01,
       -2.59222583e-01, -1.69100139e-01, -5.90072988e-01, -2.58964525e-01,
       -1.94214709e-01, -5.85067688e-01, -2.42160277e-01, -1.93750128e-01,
       -5.85595189e-01, -2.44575591e-01, -1.94594763e-01, -5.86115478e-01,
       -2.43239032e-01, -7.84925707e-02, -5.94643710e-01, -2.10017549e-01,
       -1.92422513e-01, -5.92419861e-01, -1.43077822e-01, -1.52523901e-01,
       -5.34315346e-01, -2.31594028e-01, -1.85759492e-01, -5.28855143e-01,
       -2.11206020e-01,  4.62464392e-02, -4.32122884e-01, -1.07812667e-01,
       -2.75661185e-01, -4.53846602e-01, -9.63156611e-02,  1.38283014e-01,
       -2.30549468e-01, -8.00034121e-02, -3.64339605e-01, -2.40059940e-01,
       -4.21872452e-02,  1.06035486e-01, -6.34580571e-02, -9.84783188e-02,
       -3.73050243e-01, -9.28878095e-02, -1.06062578e-01,  1.02283150e-01,
        1.39149837e-03, -

In [97]:
with open("../scaler/scaler0924.pkl", "rb") as f:
    scaler = pickle.load(f)

    normalized_data = scaler.transform([data])
    print(normalized_data)

[[  0.04949461  -1.59906898  -1.4971688    0.55740148  -1.65979108
   -1.373218     0.56093214  -1.67682385  -1.37129951   0.56809262
   -1.68708247  -1.36995122   0.07566113  -1.63572242  -1.50016923
    0.08050106  -1.64125537  -1.51357025   0.06001608  -1.64585046
   -1.50364766   2.53154794  -1.81452755  -0.78243218  -0.01693747
   -1.77389684  -1.24531834   0.58169572  -1.72383529  -1.32014652
   -0.11191848  -1.69471239  -1.49670453   4.10868831  -2.11148113
    0.29168789  -2.57097311  -2.00197296  -1.3468981    3.95383252
   -1.15247124   0.82374484  -3.8215872   -0.84358791  -0.64245889
    3.80806     -0.76156554   0.57727839  -3.71582957  -0.51621786
   -0.15290844   3.89060922  -0.43567307   0.6330218   -3.56098385
   -0.17538013  -0.12420809   3.63498675  -0.5742541    0.54338493
   -2.9045382   -0.34144253  -0.15318197   3.7173763   -0.80912548
    0.55132035  -3.31795703  -0.52487415  -0.16546283   0.
    0.           0.         -10.89245132   0.17201318  -1.47156514
   

In [8]:
with open("./model/ensemble_model.pkl", "rb") as f:
    ensemble_model = pickle.load(f)

    # print(ensemble_model.predict([data[-4:]]))
    print(ensemble_model.predict([[0.8587, 0.4786, 0.8428, 0.8630]]))


[0]


/Users/hou27/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/Users/hou27/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/Users/hou27/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/hou27/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [95]:
data[-4:]

array([142.18, 138.9 , 144.54, 160.86])

In [75]:
data[-4:]

array([142.18, 138.9 , 144.54, 160.86])